First Training Set

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Jun 21 12:23:31 2018

@author: pooja
"""

import pandas as pd
import numpy as np
import time
from math import sqrt
from numpy import random
from sklearn.metrics import mean_absolute_error,mean_squared_error





cols=['userId','itemId','rating','timestamp']
#reading data
read_start_time=time.time()
df=pd.read_table('ml-1m/ratings.dat',sep='::',header=None,names=cols)
df.drop(columns=['timestamp'],inplace=True)
read_end_time = time.time()
print("Read time = ",read_end_time-read_start_time)


# total user and movie
n_users=df.userId.max()
n_movie=df.itemId.max()



#converting data to matrix form
mat_start=time.time()
rating=np.zeros((n_users,n_movie))
for  row in df.itertuples():
    rating[row[1]-1,row[2]-1]=row[3]
mat_end=time.time()
print("Matrix conversion time = ",mat_end-mat_start)


#train_test_splitting
trainset = np.copy(rating)
testset=np.zeros((n_users,n_movie))
#z for iterate over each row
train_test=time.time()
z=0
for row in rating:
    nz_in=np.nonzero(row)
    per_20=int(len(nz_in[0])*0.2)
    rand =random.choice(nz_in[0],per_20,replace=False)
    for i in range(per_20):
        testset[z,rand[i]] =rating[z,rand[i]]
        trainset[z,rand[i]] = 0
    z =z+1

train_test_end=time.time()
print("train_test split time = ",train_test_end-train_test)
        

#adjusted cosine similarity calculation
def adj_c_sim(train_data):
    start=time.time()
    u_m = train_data.sum(axis=1)/(train_data !=0).sum(axis=1)
    rating_m_sub = np.where((train_data !=0),train_data-u_m[:,None],train_data)
    sim=np.zeros((n_movie,n_movie))
    for i in range(n_movie):
        print(i)
        #st=time.time()
        for j in range(i,n_movie):
            num=0
            dem1=0
            dem2=0
            set_c_u=np.where((train_data[:,i] !=0) * (train_data[:,j]) )[0]
            for k in set_c_u:
                num=num+rating_m_sub[k][i] * rating_m_sub[k][j]
                dem1=dem1 + rating_m_sub[k][i]**2
                dem2=dem2 + rating_m_sub[k][j]**2
                sim[i,j] = num/sqrt(dem1*dem2 +10**-12)
        #en=time.time()-st
        #print(en)
    end=time.time() -start
    print("sim time = ",end)
    return sim
                
sim=adj_c_sim(trainset)
#copying below diagonl of similarity matrix
upp_tr=np.triu(sim,k=1)
upp_tr=upp_tr.T
sim=sim+upp_tr

sim=np.where((sim <0),0,sim)
#save sim mat to sim.txt
#np.savetxt('sim.txt',sim, fmt='%.4f',delimiter=' ')

#prediction
mul=trainset.dot(sim)
div=np.zeros((n_users,n_movie))
stt=time.time()
for i in range(n_users) :
    #print(i)
    nzi=np.nonzero(trainset[i])
    #print(nzi)
    for j in range(n_movie):
        sm=(sim[j,nzi]).sum()
        div[i,j] = sm
    endd=time.time() -stt
    print(endd)
    
#np.nan_to_num(div,copy=False)
pred=mul/div
np.nan_to_num(pred,copy=False)
#save pred mat to sim.txt
#np.savetxt('pred.txt',pred, fmt='%.4f',delimiter=' ')

#Error calculation

MAE=mean_absolute_error(testset[testset!=0],pred[testset!=0])
MSE=mean_squared_error(testset[testset!=0],pred[testset!=0])
RMSE=sqrt(MSE)

print("MAE = ",MAE)
print("RMSE = ",RMSE)

#precision,Recall,F1-measure
pred_nz=pred[testset !=0]
test_nz=testset[testset !=0]
tp=0
fp=0
fn=0
th=4 #threshold value
for i in range(len(pred_nz)):
    if test_nz[i] >=th and pred_nz[i] >=th:
        tp+=1
    elif test_nz[i] < th and pred_nz[i] >= th :
        fp+=1
    elif test_nz[i] >= th and pred_nz[i] < th :
        fn+=1
precision=tp/(tp+fp)
recall=tp/(tp+fn)
#f1 measure= 2*(precision *recall)/(precision +recall)
f1_measure=2*(precision *recall)/(precision +recall)

In [ ]:
arr = np.zeros((n_users,n_movie))
for i in range(n_users):
    for j in range(n_movie):
        if testset[i,j]!=0 and pred[i,j]!=0:
            arr[i][j] = testset[i,j]
GIM = mean_absolute_error(testset[testset>3.5],pred[testset>3.5])
GPIM = mean_absolute_error(testset[arr>3.5],pred[arr>3.5])

print('GIM = ', GIM)
print('GPIM = ', GPIM)

In [ ]:
train1 = np.copy(rating)
test1=np.zeros((n_users,n_movie))
#z for iterate over each row
train_test=time.time()
z=0
for row in rating:
    nz_in=np.nonzero(row)
    per_20=int(len(nz_in[0])*0.2)
    rand =random.choice(nz_in[0],per_20,replace=False)
    for i in range(per_20):
        test1[z,rand[i]] =rating[z,rand[i]]
        train1[z,rand[i]] = 0
    z =z+1

In [ ]:
train.shape

In [ ]:
sim=adj_c_sim(train1)
#copying below diagonl of similarity matrix
upp_tr=np.triu(sim,k=1)
upp_tr=upp_tr.T
sim=sim+upp_tr

sim=np.where((sim <0),0,sim)

In [ ]:
mul=train1.dot(sim)
div=np.zeros((n_users,n_movie))
stt=time.time()
for i in range(n_users) :
    #print(i)
    nzi=np.nonzero(train1[i])
    #print(nzi)
    for j in range(n_movie):
        sm=(sim[j,nzi]).sum()
        div[i,j] = sm
    endd=time.time() -stt
    print(endd)
    
#np.nan_to_num(div,copy=False)
pred1=mul/div


In [ ]:
np.nan_to_num(pred1,copy=False)

In [ ]:
MAE1=mean_absolute_error(test1[test1!=0],pred1[test1!=0])
MSE1=mean_squared_error(test1[test1!=0],pred1[test1!=0])
RMSE1=sqrt(MSE1)

print("MAE = ",MAE1)
print("RMSE = ",RMSE1)

In [ ]:
arr1 = np.zeros((n_users,n_movie))
for i in range(n_users):
    for j in range(n_movie):
        if test1[i,j]!=0 and pred1[i,j]!=0:
            arr1[i][j] = test1[i,j]
GIM1 = mean_absolute_error(test1[test1>3.5],pred1[test1>3.5])
GPIM1 = mean_absolute_error(test1[arr1>3.5],pred1[arr1>3.5])

print('GIM = ', GIM1)
print('GPIM = ', GPIM1)

*Third Training Set*

In [ ]:
train2 = np.copy(rating)
test2=np.zeros((n_users,n_movie))
#z for iterate over each row
train_test=time.time()
z=0
for row in rating:
    nz_in=np.nonzero(row)
    per_20=int(len(nz_in[0])*0.2)
    rand =random.choice(nz_in[0],per_20,replace=False)
    for i in range(per_20):
        test2[z,rand[i]] =rating[z,rand[i]]
        train2[z,rand[i]] = 0
    z =z+1

In [ ]:
sim2=adj_c_sim(train2)
#copying below diagonl of similarity matrix
upp_tr=np.triu(sim2,k=1)
upp_tr=upp_tr.T
sim2=sim2+upp_tr

sim2=np.where((sim2 <0),0,sim2)

In [ ]:
mul=train2.dot(sim2)
div=np.zeros((n_users,n_movie))
stt=time.time()
for i in range(n_users) :
    #print(i)
    nzi=np.nonzero(train2[i])
    #print(nzi)
    for j in range(n_movie):
        sm=(sim2[j,nzi]).sum()
        div[i,j] = sm
    endd=time.time() -stt
    print(endd)
    
#np.nan_to_num(div,copy=False)
pred2=mul/div
np.nan_to_num(pred2,copy=False)

In [ ]:
MAE2=mean_absolute_error(test2[test2!=0],pred2[test2!=0])
MSE2=mean_squared_error(test2[test2!=0],pred2[test2!=0])
RMSE2=sqrt(MSE2)

print("MAE = ",MAE2)
print("RMSE = ",RMSE2)

In [ ]:
arr2 = np.zeros((n_users,n_movie))
for i in range(n_users):
    for j in range(n_movie):
        if test2[i,j]!=0 and pred2[i,j]!=0:
            arr2[i][j] = test2[i,j]
GIM2 = mean_absolute_error(test2[test2>3.5],pred2[test2>3.5])
GPIM2 = mean_absolute_error(test2[arr2>3.5],pred2[arr2>3.5])

print('GIM = ', GIM2)
print('GPIM = ', GPIM2)

Fourth Training Set

In [ ]:
train3 = np.copy(rating)
test3=np.zeros((n_users,n_movie))
#z for iterate over each row
train_test=time.time()
z=0
for row in rating:
    nz_in=np.nonzero(row)
    per_20=int(len(nz_in[0])*0.2)
    rand =random.choice(nz_in[0],per_20,replace=False)
    for i in range(per_20):
        test3[z,rand[i]] =rating[z,rand[i]]
        train3[z,rand[i]] = 0
    z =z+1

In [ ]:
sim3=adj_c_sim(train3)
#copying below diagonl of similarity matrix
upp_tr=np.triu(sim3,k=1)
upp_tr=upp_tr.T
sim3=sim3+upp_tr

sim3=np.where((sim3 <0),0,sim3)

In [ ]:
mul=train3.dot(sim3)
div=np.zeros((n_users,n_movie))
stt=time.time()
for i in range(n_users) :
    #print(i)
    nzi=np.nonzero(train3[i])
    #print(nzi)
    for j in range(n_movie):
        sm=(sim3[j,nzi]).sum()
        div[i,j] = sm
    endd=time.time() -stt
    print(endd)
    
#np.nan_to_num(div,copy=False)
pred3=mul/div
np.nan_to_num(pred3,copy=False)

In [ ]:
MAE3=mean_absolute_error(test3[test3!=0],pred3[test3!=0])
MSE3=mean_squared_error(test3[test3!=0],pred3[test3!=0])
RMSE3=sqrt(MSE3)

print("MAE = ",MAE3)
print("RMSE = ",RMSE3)

In [ ]:
arr3 = np.zeros((n_users,n_movie))
for i in range(n_users):
    for j in range(n_movie):
        if test3[i,j]!=0 and pred3[i,j]!=0:
            arr3[i][j] = test3[i,j]
GIM3 = mean_absolute_error(test3[test3>3.5],pred3[test3>3.5])
GPIM3 = mean_absolute_error(test3[arr3>3.5],pred3[arr3>3.5])

print('GIM = ', GIM3)
print('GPIM = ', GPIM3)

Fifth Training Set

In [ ]:
train4 = np.copy(rating)
test4=np.zeros((n_users,n_movie))
#z for iterate over each row
train_test=time.time()
z=0
for row in rating:
    nz_in=np.nonzero(row)
    per_20=int(len(nz_in[0])*0.2)
    rand =random.choice(nz_in[0],per_20,replace=False)
    for i in range(per_20):
        test4[z,rand[i]] =rating[z,rand[i]]
        train4[z,rand[i]] = 0
    z =z+1

In [ ]:
sim4=adj_c_sim(train4)
#copying below diagonl of similarity matrix
upp_tr=np.triu(sim4,k=1)
upp_tr=upp_tr.T
sim4=sim4+upp_tr

sim4=np.where((sim4 <0),0,sim4)

In [ ]:
mul=train4.dot(sim4)
div=np.zeros((n_users,n_movie))
stt=time.time()
for i in range(n_users) :
    #print(i)
    nzi=np.nonzero(train4[i])
    #print(nzi)
    for j in range(n_movie):
        sm=(sim4[j,nzi]).sum()
        div[i,j] = sm
    endd=time.time() -stt
    print(endd)
    
#np.nan_to_num(div,copy=False)
pred4=mul/div
np.nan_to_num(pred4,copy=False)

In [ ]:
MAE4=mean_absolute_error(test4[test4!=0],pred4[test4!=0])
MSE4=mean_squared_error(test4[test4!=0],pred4[test4!=0])
RMSE4=sqrt(MSE4)

print("MAE = ",MAE4)
print("RMSE = ",RMSE4)

In [ ]:
arr4 = np.zeros((n_users,n_movie))
for i in range(n_users):
    for j in range(n_movie):
        if test4[i,j]!=0 and pred4[i,j]!=0:
            arr4[i][j] = test4[i,j]
GIM4 = mean_absolute_error(test4[test4>3.5],pred4[test4>3.5])
GPIM4 = mean_absolute_error(test4[arr4>3.5],pred4[arr4>3.5])

print('GIM = ', GIM4)
print('GPIM = ', GPIM4)

In [ ]:
d = {'Training Set':[1,2,3,4,5],'MAE':[MAE,MAE1,MAE2,MAE3,MAE4],'RMSE':[RMSE,RMSE1,RMSE2,RMSE3,RMSE4],'GIM':[GIM,GIM1,GIM2,GIM3,GIM4],'GPIM':[GPIM,GPIM1,GPIM2,GPIM3,GPIM4]}
table = pd.DataFrame(data=d)

In [ ]:
table